# Setup

In [13]:
! pip install flask

Defaulting to user installation because normal site-packages is not writeable


In [14]:
import sys
sys.path
sys.path.append('/global/homes/r/ronitnag/.local/lib/python3.9/site-packages')
from flask import Flask, request, jsonify

In [15]:
from sqlalchemy.sql import func
#
# DESI software
import desispec.database.redshift as db
specprod = 'fuji'

# Database Setup
postgresql = db.setup_db(schema=specprod, hostname='nerscdb03.nersc.gov', username='desi')

#Flask Setup
app = Flask(__name__)

In [16]:
valid_spectypes = {'GALAXY', 'STAR', 'QSO'}
valid_subtypes = {'CV', 'M', 'G', 'K'}
default_limit = 100

# Helper Methods

In [17]:
def filter_query(q, db_ref, body, z_min=-1.0, z_max=6.0, spectype=None, subtype=None, limit=None, start=None, end=None):
    """
    Filters query based on options and provided reference table
    @Params:
        q (SQLAlchemy Query): Query object to apply filters
        db_ref (SQLAlchemy DeclarativeMeta): Table to use to apply filters (either Zpix or Ztile)
    
    @Returns:
        q (SQLAlchemy Query): Query object after filters have been applied
    """
    z_min = body.get('z_min', -1.0)
    z_max = body.get('z_max', 6.0)
    spectype = body.get('spectype', None)
    subtype = body.get('subtype', None)
    limit = body.get('limit', None)
    start = body.get('start', None)
    end = body.get('end', None)
    if (z_min > z_max):
        raise ValueError(f'z_min({z_min}) must be less than z_max({z_max})')
    if (spectype and spectype not in valid_spectypes):
        raise ValueError(f'Spectype {spectype} is not valid. Choose from available spectypes: {valid_spectypes}')
    
    if (subtype and subtype not in valid_subtypes):
        raise ValueError(f'Subtype {subtype} is not valid. Choose from available subtypes: {valid_subtypes}')
        
    if (spectype and subtype and spectype != 'STAR'):
        raise ValueError('Only STAR spectype currently have subtypes')
    
    q = q.filter(db_ref.z >= z_min).filter(db_ref.z <= z_max)
    if spectype:
        q = q.filter(db_ref.spectype == spectype)
    if subtype:
        q = q.filter(db_ref.subtype == subtype)
    
    count = q.count()
    
    if limit is not None:
        if start is not None and end is not None:
            raise ValueError('Cannot handle limit and start/end arguments to filter query')
        elif (start is not None and end is None):
            q = q.offset(start).limit(limit)
        elif (end is not None and start is None):
            if end-limit < 0:
                raise IndexError(f'Invalid end argument {end} for provided limit {limit}')
            else:
                q = q.offset(end-limit).limit(limit)
        else:
            q = q.limit(limit)
    else:
        if start is None and end is None:
            q.limit(default_limit)
        elif start is None or end is None:
            raise ValueError(f'Must provide both start and end parameters if limit is not provided')
        elif end <= start:
            raise ValueError(f'Start parameter {start} must be less than end parameter {end}')
        else:
            q = q.offset(start).limit(end-start)
    
    return q

In [18]:
def formatJSON(q):
    results = []
    for target in q.all():
        print(type(target))
        print(dict(target._mapping))
        results.append(dict(target._mapping))
    with app.app_context():
        return jsonify(results)

# API Endpoints

In [19]:
def getRedshiftByTargetID(targetID):
    """ 
    @Params: 
        targetID (BIGINT): Big Integer representing which object to query for redshift
    
    @Returns:
        z (DOUBLE): Redshift of the first object 
    """
    targetID = int(targetID)
    if (targetID < 0):
        raise ValueError(f'Target ID {targetID} is invalid')
    
    q = db.dbSession.query(db.Zpix.z).filter(db.Zpix.targetid == targetID)
    
    if (q.first() is None):
        raise ValueError(f'Target ID {targetID} was not found')
    if (q.count() > 1):
        print(f'More than one redshift value found for target: {targetID}. Returning first found')
        
    z = q[0][0]
    return z

In [20]:
getRedshiftByTargetID(39628438604088190)

-4.66664293252988e-05

In [21]:
def getRedshiftsByTileID(body):
    """ 
    @Params: 
        body (DICT): Contains query parameters.
            MUST CONTAIN: tileID, (limit/start/end)
            OPTIONAL: spectype, subtype, z_min, z_max
    
    @Returns:
        results (JSON): JSON Object (targetID, redshift) containing the targetIDs and associated 
                  redshifts for targets found in provided tileID.     
    """
    # body = request.get_json()
    tileID = body['tileID']
    
    if (tileID < 1):
        raise ValueError(f'Tile ID {tileID} is invalid')                         
  
    q = db.dbSession.query(db.Ztile.targetid, db.Ztile.z).filter(db.Ztile.tileid == tileID)
    
    if (q.first() is None):
        raise ValueError(f'Tile ID {tileID} was not found')
    
    q = filter_query(q, db.Ztile, body)
    return formatJSON(q)

In [22]:
r = getRedshiftsByTileID({"tileID":330, "limit":10})

<class 'sqlalchemy.engine.row.Row'>
{'targetid': 39633149646341246, 'z': 0.840532532904315}
<class 'sqlalchemy.engine.row.Row'>
{'targetid': 39633123209643991, 'z': -0.00199569129234795}
<class 'sqlalchemy.engine.row.Row'>
{'targetid': 39633145267490337, 'z': 0.000441978242000794}
<class 'sqlalchemy.engine.row.Row'>
{'targetid': 39633145271681767, 'z': -0.00199569129234795}
<class 'sqlalchemy.engine.row.Row'>
{'targetid': 39633149637951967, 'z': 1.2497758987796}
<class 'sqlalchemy.engine.row.Row'>
{'targetid': 39633149637952037, 'z': 0.755642641997885}
<class 'sqlalchemy.engine.row.Row'>
{'targetid': 39633145267489278, 'z': 0.986790786461522}
<class 'sqlalchemy.engine.row.Row'>
{'targetid': 39633149637953471, 'z': 1.37466122192437}
<class 'sqlalchemy.engine.row.Row'>
{'targetid': 39633145267489903, 'z': 1.4928476773864}
<class 'sqlalchemy.engine.row.Row'>
{'targetid': 39633149637951997, 'z': 1.35234920490745}


In [25]:
r.get_data()

b'[{"targetid":39633149646341246,"z":0.840532532904315},{"targetid":39633123209643991,"z":-0.00199569129234795},{"targetid":39633145267490337,"z":0.000441978242000794},{"targetid":39633145271681767,"z":-0.00199569129234795},{"targetid":39633149637951967,"z":1.2497758987796},{"targetid":39633149637952037,"z":0.755642641997885},{"targetid":39633145267489278,"z":0.986790786461522},{"targetid":39633149637953471,"z":1.37466122192437},{"targetid":39633145267489903,"z":1.4928476773864},{"targetid":39633149637951997,"z":1.35234920490745}]\n'